# Setup

In [6]:
import pickle
import tensorflow as tf
import os

tf.test.is_gpu_available()


False

In [ ]:
from biomedical_qa import util

from biomedical_qa.inference.inference import Inferrer, get_model_and_session
from biomedical_qa.sampling.bioasq import BioAsqSampler

import matplotlib.pyplot as plt
%pylab inline
pylab.rcParams['figure.figsize'] = (10, 6)

print("Done!")

In [5]:
import os
os.environ["CUDA_DEVICE_ORDER"]= "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
CONFIG = "./results/config.pickle"
#"../model_checkpoints/simple_pointer_with_chars/bioasq_finetune_dr05_sigmoid_bioasqtrainer_ckptits200/config.pickle"

model, sess = get_model_and_session(CONFIG, ["cpu:0"])
inferrer = Inferrer(model, sess, beam_size=1)

NameError: name 'get_model_and_session' is not defined

In [ ]:
vocab = inferrer.model.embedder.vocab
rev_vocab = [""] * len(vocab)
for w, i in vocab.items():
    rev_vocab[i] = w



In [ ]:
sampler = BioAsqSampler("../data/bioasq", ["dev.json"], 16, vocab=vocab, shuffle=False)

# Run Model

In [ ]:
predictions = inferrer.get_predictions(sampler)

# Factoid Analysis

In [ ]:
factoid_predictions = {id: p for id, p in predictions.items() if p.question.q_type == "factoid"}

In [ ]:
for prediction in factoid_predictions.values():
    
    print_prediction(prediction, rev_vocab)

In [ ]:
def is_correct_lenient(prediction):
    correct_answers = maybe_flatten_list(prediction.question.question_json["original_answers"])
    for correct_answer in correct_answers:
        for predicted_answer in prediction.answer_strings:
            if correct_answer.lower().strip() == predicted_answer.lower().strip():
                return True
    return False

In [ ]:
correct_predictions = {id: p for id, p in factoid_predictions.items() if is_correct_lenient(p)}
incorrect_predictions = {id: p for id, p in factoid_predictions.items() if not is_correct_lenient(p)}

print("Correct: %d, Incorrect: %d" % (len(correct_predictions), len(incorrect_predictions)))

## Pointer Visualization (Incorrect Answer)

In [ ]:
for prediction in incorrect_predictions.values():
    print_prediction(prediction, rev_vocab)

In [ ]:
prediction = incorrect_predictions["54fc99f36ad7dcbc12000004"]

In [ ]:
print_prediction(prediction, rev_vocab)

In [ ]:
tokens = get_tokens(prediction, rev_vocab)

In [ ]:
correct_tokens = find_correct_tokens(prediction.question)

text_heatmap(tokens, prediction.prediction.start_probs, correct_tokens)

In [ ]:
# TODO
#start_token_mask = np.zeros(len(tokens), dtype=np.bool)
#start_token_mask[prediction.starts[0]] = True

#text_heatmap(tokens, prediction.end_scores[0], start_token_mask)